In [20]:
import urllib.request
from bs4 import BeautifulSoup
import requests
import re
from tika import parser
from tqdm import tqdm
import pandas as pd
import itertools
import numpy as np
from collections import Counter
import datetime

In [21]:
input = '1   1 1   1'

input = input.split(" ")
c = Counter(input)
s = "".join(c.keys())


In [22]:
s_double = [key for key, value in c.items() if value==4 and key!='']*2

In [23]:
def remov_duplicates(input):
    input = input.split(" ")
    c = Counter(input)
    s = "".join(c.keys())
    s_double = [key for key, value in c.items() if value==4 and key!='']*2
    if s_double:
        s="".join(s_double)
    return(s)

def has_numbers(input):
    digit_count = sum(char.isdigit() for char in input)
    if digit_count>1: 
        return True 
    else: 
        return False
    
def clean_data(i):
    i = re.sub("[5-9]\s+P M", "", i)
    i = re.sub('[A-Z]','', i)
    i = i.replace(""," ")
    i = remov_duplicates(i)
    return(i)

In [24]:
pdf = './raw_data/NYCB_2023 Fall Season.pdf'
rawText = parser.from_file(pdf)

In [25]:
exclude = ['75TH ANNIVERSARY']

rawList = rawText['content'].splitlines()
rawList = [i for i in rawList if len(i)>0 and i not in exclude]

In [26]:
### join names that are split
join_ind = []
exceptions = ['Tarantella ']
for ind, i in enumerate(rawList): 
    if i.endswith(' ') and i not in exceptions and not has_numbers(i):
        join_ind.append(ind)
        join_ind.append(ind+1)

l = sorted(set(join_ind))
group = []
val = 0
for ind, i in enumerate(l):
    if ind==0:
        group.append(val)
    else:
        if l[ind] - l[ind-1] > 1:
            val += 1
        group.append(val)

g_df = pd.DataFrame({'ind':l, 'group':group})
ind_to_pop = []
for ind in range(g_df.shape[0]):
    min_ind = g_df[g_df.group==ind].ind.min()
    max_ind = g_df[g_df.group==ind].ind.max()+1
    if not np.isnan(min_ind):
        new_string = ''.join(rawList[min_ind:max_ind])
        rawList[min_ind] = new_string
        ind_to_pop.append([i for i in range(min_ind+1, max_ind, 1)])

ind_to_pop = list(itertools.chain(*ind_to_pop))
rawList = [i for j, i in enumerate(rawList) if j not in ind_to_pop]

In [27]:
days = ['TUES', 'WED', 'THUR', 'FRI', 'SAT - MAT', 'SAT - EVE', 'SUN']

In [28]:
rawList = rawList[rawList.index('FALL SEPT-OCT 2023')+1:-1]

In [29]:
dates = []
dates_ind = []
for ind, i in enumerate(rawList):
    if has_numbers(i):
        dates.append(clean_data(i))
        dates_ind.append(ind)

In [30]:
program_groups = sorted([i for i in range(0, len(dates), 1)] * round(len(dates)/7))
program_groups_ind = [i for i in range(0, 4, 1)] * len(program_groups)

In [31]:
M = round(len(dates)/7)  #columns
N = 7 #rows
matrix = [[0] * N for _ in range(M)]

In [32]:
# for ind, g, g_ind in zip(range(len(dates_ind)), program_groups, program_groups_ind):
#     min_ind = dates_ind[ind]+1
#     # try:
#     max_ind = dates_ind[ind+1]
# # print(ind)
# # print(rawList[min_ind:max_ind])
#     print(g, g_ind)
#     matrix[g_ind][g] = rawList[min_ind:max_ind]
#     # except:
#     #     continue

# pd.DataFrame(matrix)


In [33]:
program_names = []
pieces = []
for ind in range(len(dates_ind)):
    min_ind = dates_ind[ind]+1
    try:
        max_ind = dates_ind[ind+1]
    except:
        max_ind = -1
    subset = rawList[min_ind:max_ind]
    program_name = "".join([i for i in subset if i.isupper()])
    s = ", ".join([re.sub(' +', ' ', i).strip() for i in subset if i not in program_name])
    program_names.append(program_name)
    pieces.append(s)


In [34]:
# date, day of week, program title, pieces

In [35]:
fall_months = ['SEPT', 'OCT']

In [36]:
d = pd.DataFrame({
    'date':dates,
    'day':list(np.repeat(days, 4)),
    'program name':program_names,
    'repertory':pieces
})

In [37]:
d[d.day=='SAT - MAT']

,date,day,program name,repertory
16,30,SAT - MAT,ALL BALANCHINE II,"Bourrée Fantasque, Agon, Slaughter on Tenth Av..."
17,23,SAT - MAT,JEWELS,
18,7,SAT - MAT,ALL BALANCHINE III,"Apollo, La Sonnambula, Tschaikovsky Piano Conc..."
19,14,SAT - MAT,ALL BALANCHINE V,"Serenade, Orpheus, Theme and Variations"


In [38]:
month_lis = []
for day in days:
    l = d[d.day==day].date.tolist()
    month_ind = 0
    for i in range(len(l)):
        if int(l[i-1])-int(l[i])>15:
            month_ind += 1
        month_lis.append(fall_months[month_ind])

d['month'] = month_lis
d['month_date'] = [sub[:3] for sub in d['month']]+d['date']
d['date'] = [datetime.datetime.strptime(i, '%b%d').strftime('%b %d') for i in d['month_date'].tolist()]
d = d.drop(['month', 'month_date'],axis=1)

In [ ]:
# save to CSV and do for each!

In [ ]:
### LINKS TO YOUTUBE VIDEOS SHOWING/DISCUSSING THESE PIECES THIS SEASON?

## next steps -- need to search NYCB rep page for info on music, etc. more details on the piece